In [2]:
!pip install -q transformers
!pip install -q torchinfo
!pip install -U -q datasets fsspec huggingface_hub # Hugging Face's dataset library
!pip install -q datasets
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.

In [10]:
import numpy as np
import pandas as pd
import transformers
import evaluate
from datasets import load_dataset
from torchinfo import summary
from transformers import BertTokenizer, BertModel, BertForSequenceClassification,AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
import torch.nn as nn
import torch
import random
from itertools import islice
import torch.nn.functional as F
from torch.nn import CosineSimilarity
import nltk
from nltk.tokenize import sent_tokenize
from torch.nn import MSELoss, CosineSimilarity
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
import torch, torch.nn as nn, torch.nn.functional as F
from collections import defaultdict
from torch.utils.data import DataLoader, IterableDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# For sentence processing later
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [5]:
def load_models():
    bert_model = BertModel.from_pretrained("bert-base-uncased")
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    llama_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name)
    llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
    bert_model.eval()
    llama_model.eval()
    return bert_model, bert_tokenizer, llama_model, llama_tokenizer

In [6]:
bert, bert_tokenizer, llama, llama_tokenizer = load_models()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def encode_sentence(sentence, tokenizer, max_len=20):
    return tokenizer(
        sentence,
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

In [11]:
# llama_embed_dim = llama.get_input_embeddings().weight.size(-1)
# llama_embed_dim

2048

In [8]:
def train_projection_layer(
    bert, llama, projection, data_stream,
    tokenizer_bert, tokenizer_llama,
    optimizer, loss_fn, max_steps=5000,
    log_interval=50, checkpoint_interval=1000, device=None
):
    if device is None:
        device = next(projection.parameters()).device
    step = 0
    running_loss = 0.0
    running_cosine = 0.0
    cosine_fn = nn.CosineSimilarity(dim=1)

    for example in data_stream:
        text = example.get("text", "").strip()
        if not text:
            continue

        # Get first sentence
        sentence = sent_tokenize(text)[0] if len(sent_tokenize(text)) > 0 else ""
        if not sentence:
            continue

        # Encode with BERT
        bert_inputs = {k: v.to(device) for k, v in
               encode_sentence(sentence, tokenizer_bert).items()}
        with torch.no_grad():
            output = bert(**bert_inputs)
            token_embeddings = output.last_hidden_state
            attention_mask = bert_inputs['attention_mask'].unsqueeze(-1)
            masked_output = token_embeddings * attention_mask
            pooled = masked_output.sum(dim=1) / attention_mask.sum(dim=1)
            pooled = pooled.to(device)


        # Encode with LLaMA
        llama_inputs = {k: v.to(device)
                for k, v in encode_sentence(sentence, tokenizer_llama).items()}

        llama_input_ids  = llama_inputs["input_ids"]
        llama_input_embed = llama.get_input_embeddings()(llama_input_ids)

        if llama_input_embed.shape[1] < 2:
            continue

        target_embedding = llama_input_embed[:, 1, :].detach()
        projected = projection(pooled)

        # Compute loss
        mse_loss = loss_fn(projected, target_embedding)
        cosine_sim = cosine_fn(projected, target_embedding)
        loss = mse_loss - 0.5 * cosine_sim.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute cosine similarity
        cosine_sim = cosine_fn(projected, target_embedding).mean().item()

        # Track metrics
        running_loss += loss.item()
        running_cosine += cosine_sim

        if step % log_interval == 0:
            avg_loss = running_loss / (log_interval if step > 0 else 1)
            avg_cos = running_cosine / (log_interval if step > 0 else 1)
            print(f"Step {step}: Avg MSE = {avg_loss:.4f} | Avg Cosine = {avg_cos:.4f}")
            running_loss = 0.0
            running_cosine = 0.0

        if step > 0 and step % checkpoint_interval == 0:
            print("Saving model checkpoint...")
            torch.save({
                'step': step,
                'model_state_dict': projection.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
                'cosine_similarity': cosine_sim,
            }, f"projection_checkpoint_step_{step}.pth")

        step += 1
        if step >= max_steps:
            break

In [9]:
def generate_text_from_prompt(prompt, bert, llama, projection,
                              tokenizer_bert, tokenizer_llama,
                              max_new_tokens=50, top_p=0.9, temperature=1.0):

    device = next(bert.parameters()).device
    bert_inputs = {k: v.to(device) for k, v in
                   encode_sentence(prompt, tokenizer_bert).items()}

    with torch.no_grad():
        bert_output = bert(**bert_inputs)
        token_emb   = bert_output.last_hidden_state
        mask        = bert_inputs['attention_mask'].unsqueeze(-1)
        pooled      = (token_emb * mask).sum(dim=1) / mask.sum(dim=1)
        projected   = projection(pooled)

    input_ids = tokenizer_llama(prompt, return_tensors="pt")["input_ids"].to(device)


    input_emb = llama.get_input_embeddings()(input_ids)
    if input_emb.shape[1] >= 2:
        input_emb[:, 1, :] = projected
    else:
        print("Warning: Prompt too short, skipping token replacement.")

    # Step 3: Generate text using top-p sampling
    with torch.no_grad():
        outputs = llama.generate(
            inputs_embeds=input_emb,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
            eos_token_id=tokenizer_llama.eos_token_id,
            pad_token_id=tokenizer_llama.pad_token_id
        )

    generated_text = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    print(f"\n Prompt: {prompt}\n Generated: {generated_text[len(prompt):].strip()}\n")


In [13]:
def identity_cosine(llama, llama_tokenizer, device, n_sent=1000):
    embed_dim = llama.get_input_embeddings().weight.size(-1)
    eye_proj  = nn.Linear(embed_dim, embed_dim, bias=False).to(device)
    with torch.no_grad(): eye_proj.weight.copy_(torch.eye(embed_dim, device=device))
    dataset = load_dataset("c4", "en", split="train", streaming=True)
    cos_fn = nn.CosineSimilarity(dim=-1); total, k = 0., 0
    llama.eval()
    for ex in dataset.take(n_sent):
        sent = ex["text"].split(".")[0]
        ids  = llama_tokenizer(sent, return_tensors="pt").input_ids.to(device)
        tgt  = llama.get_input_embeddings()(ids)[:, 1, :]
        out  = eye_proj(tgt)
        cos  = cos_fn(F.normalize(out,-1), F.normalize(tgt,-1)).mean().item()
        total += cos; k += 1
    print(f"[Identity check] Avg cosine ≈ {total/k:.4f}")


def procrustes_rotation(dataset, num_pairs, bert, bert_tok,
                         llama, llama_tok, device, hidden_out):
    X, Y = [], []
    with torch.no_grad():
        for ex in dataset.take(num_pairs):
            sent = ex["text"].split(".")[0]
            # BERT pooled
            b_ids = bert_tok(sent, return_tensors="pt").to(device)
            b_out = bert(**b_ids).last_hidden_state
            mask  = b_ids["attention_mask"].unsqueeze(-1)
            pooled = (b_out*mask).sum(1)/mask.sum(1)              # (1,768)
            # Llama token‑1 embed
            l_ids = llama_tok(sent, return_tensors="pt").input_ids.to(device)
            l_emb = llama.get_input_embeddings()(l_ids)[:,1,:]    # (1,4096)
            X.append(pooled.squeeze().cpu().numpy())
            Y.append(l_emb.squeeze().cpu().numpy())
    X, Y = np.stack(X), np.stack(Y)

    Xp, Yp = PCA(256).fit_transform(X), PCA(256).fit_transform(Y)
    U, _, Vt = np.linalg.svd(Yp.T @ Xp)
    R = (U @ Vt)[:hidden_out, :]                                # slice if needed
    return torch.tensor(R, dtype=torch.float32)

def load_rotation_into_projection(R, projection):
    with torch.no_grad():
        projection[0].weight.copy_(R)


def plot_cosine_vs_len(n_samples, bert, bert_tok, llama, llama_tok,
                       projection, device):
    buckets = defaultdict(list); cos_fn = nn.CosineSimilarity(dim=-1)
    with torch.no_grad():
        for ex in dataset.take(n_samples):
            sent    = ex["text"].split(".")[0]
            length  = len(sent.split())
            # embeddings
            b_ids = bert_tok(sent, return_tensors="pt").to(device)
            b_out = bert(**b_ids).last_hidden_state
            mask  = b_ids["attention_mask"].unsqueeze(-1)
            pooled = (b_out*mask).sum(1)/mask.sum(1)
            proj   = projection(pooled)
            l_ids  = llama_tok(sent, return_tensors="pt").input_ids.to(device)
            tgt    = llama.get_input_embeddings()(l_ids)[:,1,:]
            cosine = cos_fn(F.normalize(proj,-1), F.normalize(tgt,-1)).item()
            buckets[length].append(cosine)
    xs = sorted(buckets); ys = [np.mean(buckets[k]) for k in xs]
    plt.plot(xs, ys, marker="o"); plt.xlabel("# tokens"); plt.ylabel("cosine")
    plt.title("Cosine vs prompt length"); plt.show()


def alpha_grid(alphas, steps, train_fn):
    results = {}
    for a in alphas:
        loss_fn = lambda p,t: F.mse_loss(p,t) - a*F.cosine_similarity(p,t,dim=1).mean()
        val_cos = train_fn(loss_fn, steps)
        results[a] = val_cos
    print("α‑grid results:", results)

def batch_sweep(exp_list, build_loader_fn, train_epoch_fn, eval_fn):
    logs = []
    for e in exp_list:
        eff_bsz = 2**e
        loader  = build_loader_fn(eff_bsz)
        train_epoch_fn(loader)
        logs.append((math.log2(eff_bsz), eval_fn()))
    xs, ys = zip(*logs)
    plt.plot(xs, ys, marker="x"); plt.xlabel("log2(batch)"); plt.ylabel("cosine")
    plt.title("Cosine vs batch size"); plt.show()


In [ ]:
def main():
    set_seed()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    bert,  bert_tokenizer,  llama,  llama_tokenizer = load_models()
    bert.to(device)
    llama.to(device)


    dataset = load_dataset("c4", "en", split="train", streaming=True)


    identity_cosine(llama, llama_tokenizer, device, n_sent=1000)

    # Initialize projection layer and optimizer
    llama_embed_dim = llama.get_input_embeddings().weight.size(-1)
    hidden_1 = 2048

    projection = nn.Sequential(
        nn.LayerNorm(768),
        nn.Linear(768, hidden_1),     # 768 → 2048
        nn.ReLU(),
        nn.LayerNorm(hidden_1),       # (→ 2048)

        nn.Linear(hidden_1, llama_embed_dim),# 2048 → 4096
        nn.ReLU(),
        nn.LayerNorm(llama_embed_dim),       # (→ 4096)

        nn.Linear(llama_embed_dim, llama_embed_dim)
    ).to(device)
    # projection = nn.Sequential(
    #     nn.Linear(768, 2048),
    #     nn.ReLU(),
    #     nn.Linear(2048, llama_embed_dim),   # <- use llama_embed_dim here
    #     nn.LayerNorm(llama_embed_dim)
    # ).to(device)

    optimizer = torch.optim.Adam(projection.parameters(), lr=1e-6)


    def combined_loss_fn(projected, target):
        mse = MSELoss()(projected, target)
        cosine = CosineSimilarity(dim=1)(projected, target).mean()
        return mse - 0.9 * cosine

    loss_fn = combined_loss_fn

    # Train projection layer
    train_projection_layer(
        bert, llama, projection,
        dataset, bert_tokenizer, llama_tokenizer,
        optimizer, loss_fn,
        max_steps=100000, log_interval=1000
    )

    # Save the trained projection layer
    torch.save(projection.state_dict(), "projection_layer.pt")
    print("Training complete. Projection layer saved to 'projection_layer.pt'.")

        # Generate text using the trained projection
    test_prompts = [
        "The future of AI in medicine",
        "Climate change will impact",
        "In 10 years, robots will",
        "Democracy and technology are",
        "Humanity's next big leap is"
    ]
    for prompt in test_prompts:
        generate_text_from_prompt(
            prompt,
            bert, llama, projection,
            bert_tokenizer, llama_tokenizer,
            max_new_tokens=50,
            top_p=0.9,
            temperature=0.7
        )

if __name__ == "__main__":
    main()

[Identity check] Avg cosine ≈ 1.0000
Step 0: Avg MSE = 0.3405 | Avg Cosine = -0.0038
Step 1000: Avg MSE = 0.1464 | Avg Cosine = 0.0754
Saving model checkpoint...
Step 2000: Avg MSE = -0.0673 | Avg Cosine = 0.1709
Saving model checkpoint...
Step 3000: Avg MSE = -0.1585 | Avg Cosine = 0.2077
Saving model checkpoint...
Step 4000: Avg MSE = -0.2202 | Avg Cosine = 0.2317
Saving model checkpoint...
Step 5000: Avg MSE = -0.2472 | Avg Cosine = 0.2374
Saving model checkpoint...
Step 6000: Avg MSE = -0.2657 | Avg Cosine = 0.2417
Saving model checkpoint...
Step 7000: Avg MSE = -0.2833 | Avg Cosine = 0.2477
Saving model checkpoint...
